In [1]:
import collections
import contextlib
import sys
import wave
import webrtcvad
import numpy as np
import pyaudio
from time import monotonic
import malaya_speech
import numpy as np
from malaya_speech import Pipeline

C:\Users\developer\Desktop\uluc_nemo\uluc_nemo\lib\site-packages\malaya_boilerplate\frozen_graph.py:35: UserWarning: Cannot import beam_search_ops from Tensorflow Addons, `deep_model` for stemmer will not available to use, make sure Tensorflow Addons version >= 0.12.0
  'Cannot import beam_search_ops from Tensorflow Addons, `deep_model` for stemmer will not available to use, make sure Tensorflow Addons version >= 0.12.0'


In [2]:
def write_wave(path, audio, sample_rate):
    """Writes a .wav file.
    Takes path, PCM audio data, and sample rate.
    """
    with contextlib.closing(wave.open(path, 'wb')) as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(sample_rate)
        wf.writeframes(audio)

    return audio

def has_speech_activity(frames,number_of_frames = 25,percentage=0.85):
	# print("len_frames: ",len(frames))
	if len(frames) < number_of_frames:
		return False

	num_voiced = len([1 for frame in frames[-number_of_frames:] if vad.is_speech(frame, RATE)])
	print("num_voiced: ",num_voiced,end='\r')
	return num_voiced > percentage * number_of_frames

In [45]:
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
import wave
from datetime import datetime

VAD_ROBUSTNESS = 1
vad = webrtcvad.Vad(VAD_ROBUSTNESS)

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print(p.get_device_info_by_index(1))


MIN_CONSECUTIVE_FRAMES = 25
MAX_WAIT = 2


for i in range(1):
	consecutive_frames_reached = 0
	frames = []
	concat_last_frame = []
	int_last_frames = []
	last_frames = []
	has_spoken = False
	byte_array = b""

	silent_time = None


	while True:
	    raw = stream.read(CHUNK, exception_on_overflow=False)
	    byte_array+= raw
# 	    filename_temp = datetime.now().strftime('vad_test_file.wav')
# 	    wf = wave.open(filename_temp, 'wb')
            
# 	    in_data = wf.readframes(raw)
       
	    _last_frame = np.fromstring(raw, dtype=np.int16)
# 	    concat_last_frame = np.concatenate((concat_last_frame,raw))
	    pcm = _last_frame.tostring()

	    if has_speech_activity(last_frames):
	        silent_time = None
	        print("Consecutive frames detected!!!")
	        has_spoken = True
	    else:
	        if silent_time is None:
	            silent_time = monotonic()
	        elif monotonic() - silent_time > MAX_WAIT and has_spoken:
	            print("Voice command ended!!!")
	            y = write_wave(f"test_{i}.wav",b''.join([f for f in frames]),RATE)
	            break
	    last_frames.append(_last_frame[0:480].tostring())
	    int_last_frames.append(_last_frame)
	    frames.append(pcm)
# int_frames = np.frombuffer(last_frames, dtype=np.uint8, count=3)



{'index': 1, 'structVersion': 2, 'name': 'Microphone Array (Realtek(R) Au', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}


C:\Users\developer\Desktop\uluc_nemo\uluc_nemo\lib\site-packages\ipykernel_launcher.py:46: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
C:\Users\developer\Desktop\uluc_nemo\uluc_nemo\lib\site-packages\ipykernel_launcher.py:48: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
C:\Users\developer\Desktop\uluc_nemo\uluc_nemo\lib\site-packages\ipykernel_launcher.py:61: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.


Consecutive frames detected!!!
Consecutive frames detected!!!
Consecutive frames detected!!!
Consecutive frames detected!!!
Consecutive frames detected!!!
Consecutive frames detected!!!
Consecutive frames detected!!!
Consecutive frames detected!!!
Consecutive frames detected!!!
Consecutive frames detected!!!
Consecutive frames detected!!!
Consecutive frames detected!!!
Consecutive frames detected!!!
Consecutive frames detected!!!
Consecutive frames detected!!!
Consecutive frames detected!!!
Voice command ended!!!


In [25]:
def byte_to_float(byte):
    # byte -> int16(PCM_16) -> float32
    return pcm2float(np.frombuffer(byte,dtype=np.int16), dtype='float32')

def pcm2float(sig, dtype='float32'):
    """Convert PCM signal to floating point with a range from -1 to 1.
    Use dtype='float32' for single precision.
    Parameters
    ----------
    sig : array_like
        Input array, must have integral type.
    dtype : data type, optional
        Desired (floating point) data type.
    Returns
    -------
    numpy.ndarray
        Normalized floating point data.
    See Also
    --------
    float2pcm, dtype
    """
    sig = np.asarray(sig)
    if sig.dtype.kind not in 'iu':
        raise TypeError("'sig' must be an array of integers")
    dtype = np.dtype(dtype)
    if dtype.kind != 'f':
        raise TypeError("'dtype' must be a floating point type")

    i = np.iinfo(sig.dtype)
    abs_max = 2 ** (i.bits - 1)
    offset = i.min + abs_max
    return (sig.astype(dtype) - offset) / abs_max
byte_to_float(_last_frame)

array([-0.00094604, -0.00094604, -0.0010376 , ..., -0.00030518,
       -0.00030518, -0.0005188 ], dtype=float32)

In [33]:
_last_frame
np.concatenate((_last_frame,_last_frame)).shape


(2048,)

In [46]:
sr = 16000
sf.write("test.wav",byte_to_float(byte_array),sr,'PCM_16')

In [20]:
np.fromstring(raw, dtype=np.float32)

C:\Users\developer\Desktop\uluc_nemo\uluc_nemo\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  """Entry point for launching an IPython kernel.


array([         nan,          nan,          nan,          nan,
                nan,          nan,          nan,          nan,
                nan,          nan,          nan,          nan,
                nan,          nan,          nan,          nan,
                nan,          nan,          nan,          nan,
                nan,          nan,          nan, 9.183550e-41,
                nan,          nan,          nan,          nan,
       3.673504e-40, 2.755093e-40, 2.755135e-40,          nan,
       3.673406e-40,          nan,          nan,          nan,
                nan,          nan,          nan,          nan,
                nan,          nan,          nan,          nan,
                nan,          nan,          nan,          nan,
                nan,          nan,          nan,          nan,
       5.510088e-40, 4.591859e-40, 9.184951e-41, 6.428527e-40,
       5.605194e-45, 7.346938e-40, 1.102034e-39, 1.193881e-39,
       4.591859e-40, 1.285712e-39,          nan, 8.2653

In [18]:
import soundfile as sf
data, old_samplerate = sf.read('test_4.wav')
data.shape

(84992,)

In [47]:
y, sr = malaya_speech.load('test.wav')

y

array([-0.0015456 , -0.0010304 , -0.0010304 , ...,  0.0010304 ,
        0.00206079,  0.0005152 ])

In [50]:
!pip install webrtcvad
!pip install malaya-speech

RATE = 16000

y_int = malaya_speech.astype.float_to_int(y)
vad = malaya_speech.vad.webrtc(minimum_amplitude=int(np.quantile(np.abs(y_int), 0.2)))
malaya_speech.vad.available_model()
model_factor1 = malaya_speech.vad.deep_model(model='marblenet-factor1')
frames_int = list(malaya_speech.utils.generator.frames(y_int, 30, RATE))


frames = list(malaya_speech.utils.generator.frames(y, 30, RATE))

frames_deep_factor1 = [(frame, model_factor1(frame)) for frame in frames]


In [62]:
model_factor1(frame)

NameError: name 'frame' is not defined

In [59]:
!pip install networkx
p = Pipeline()
pipeline = (
	p.batching(5)
		.foreach_map(model_factor1.predict)
		.flatten()
)

result = p.emit(frames)
result.keys()

frames_deep_factor1_batch = [(frame, result['flatten'][no]) for no, frame in enumerate(frames)]
len(frames_deep_factor1) == len(frames_deep_factor1_batch)

malaya_speech.extra.visualization.visualize_vad(y, frames_deep_factor1, RATE)

ExecutableNotFound: failed to execute 'dot', make sure the Graphviz executables are on your systems' PATH

In [61]:
import collections
import contextlib
import sys
import wave
import webrtcvad
import numpy as np
import pyaudio
from time import monotonic
import malaya_speech
import numpy as np
from malaya_speech import Pipeline
import IPython.display as ipd
import soundfile as sf

def write_wave(path, audio, sample_rate):
    """Writes a .wav file.
    Takes path, PCM audio data, and sample rate.
    """
    with contextlib.closing(wave.open(path, 'wb')) as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(sample_rate)
        wf.writeframes(audio)
    return audio

def byte_to_float(byte):
    # byte -> int16(PCM_16) -> float32
    return pcm2float(np.frombuffer(byte,dtype=np.int16), dtype='float32')

def pcm2float(sig, dtype='float32'):
    """Convert PCM signal to floating point with a range from -1 to 1.
    Use dtype='float32' for single precision.
    Parameters
    ----------
    sig : array_like
        Input array, must have integral type.
    dtype : data type, optional
        Desired (floating point) data type.
    Returns
    -------
    numpy.ndarray
        Normalized floating point data.
    See Also
    --------
    float2pcm, dtype
    """
    sig = np.asarray(sig)
    if sig.dtype.kind not in 'iu':
        raise TypeError("'sig' must be an array of integers")
    dtype = np.dtype(dtype)
    if dtype.kind != 'f':
        raise TypeError("'dtype' must be a floating point type")

    i = np.iinfo(sig.dtype)
    abs_max = 2 ** (i.bits - 1)
    offset = i.min + abs_max
    return (sig.astype(dtype) - offset) / abs_max


CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
import wave

VAD_ROBUSTNESS = 1
vad = webrtcvad.Vad(VAD_ROBUSTNESS)

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print(p.get_device_info_by_index(1))

MIN_CONSECUTIVE_FRAMES = 25
MAX_WAIT = 2


def has_speech_activity(frames, _last_frame ,number_of_frames = 25,percentage=0.85):
	# print("len_frames: ",len(frames))
	if len(frames) < number_of_frames:
		return False
    
	y = byte_to_float(_last_frame)
    
	y_int = malaya_speech.astype.float_to_int(y)
    
	malaya_speech.vad.available_model()
	model_factor1 = malaya_speech.vad.deep_model(model='marblenet-factor1')
	frames_int = list(malaya_speech.utils.generator.frames(y_int, 30, RATE))
    
	frames = list(malaya_speech.utils.generator.frames(y, 30, RATE))
    
	frames_deep_factor1 = [(frame, model_factor1(frame)) for frame in frames]
    
    

	num_voiced = len([1 for frame in frames[-number_of_frames:] if vad.is_speech(frame, RATE)])
	print("num_voiced: ",num_voiced,end='\r')
	return num_voiced > percentage * number_of_frames






for i in range(1):
    consecutive_frames_reached = 0
    frames = []
    concat_last_frame = []
    int_last_frames = []
    last_frames = []
    has_spoken = False
    byte_array = b""

    silent_time = None

    while True:
        raw = stream.read(CHUNK, exception_on_overflow=False)
        byte_array += raw

        _last_frame = np.fromstring(raw, dtype=np.int16)
        pcm = _last_frame.tostring()


        if has_speech_activity(last_frames, _last_frame):
            silent_time = None
            print("Consecutive frames detected!!!")
            has_spoken = True
        else:
            if silent_time is None:
                silent_time = monotonic()
            elif monotonic() - silent_time > MAX_WAIT and has_spoken:
                print("Voice command ended!!!")
                y = write_wave(f"test_{i}.wav", b''.join([f for f in frames]), RATE)
                print(frames)
                break
        last_frames.append(_last_frame[0:480].tostring())
        int_last_frames.append(_last_frame)
        frames.append(pcm)



# sf.write("test.wav",byte_to_float(byte_array),RATE,'PCM_16')
#   y = y / (np.max(np.abs(y)) + 1e-9)
# y, RATE = malaya_speech.load('test.wav')



# p = Pipeline()
# pipeline = (
#     p.batching(5)
#     .foreach_map(model_factor1.predict)
#     .flatten()
# )
# # p.visualize()
#
# result = p.emit(frames)
# result.keys()
#
#
# frames_deep_factor1_batch = [(frame, result['flatten'][no]) for no, frame in enumerate(frames)]
# len(frames_deep_factor1) == len(frames_deep_factor1_batch)
#
# probs = [(frame, model_factor1.predict_proba([frame])) for frame in frames]
# probs[:5]
#
# # malaya_speech.extra.visualization.visualize_vad(y, frames_deep_factor1, RATE)
#
# frames_vad = [
#     (frame, result['flatten'][no]) for no, frame in enumerate(frames)
# ]
#
# splitted = malaya_speech.split.split_vad(frames_vad,  negative_threshold = 0.1)
#
# ipd.Audio(splitted[0].array, rate = RATE)
#
# sf.write("output.wav", splitted[0].array, RATE, 'PCM_16')


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 88)